In [1]:
import random
import math

In [2]:
import csv
x = []
y = []
vx = []
vy = []
with open('ce889_dataCollectionN.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    for line in csv_reader:
        x.append(float(line[0]))
        y.append(float(line[1]))
        vx.append(float(line[2]))
        vy.append(float(line[3]))

In [3]:
class neuron:
    
    def multWeights(self, prev_layer, index):
        total = 0
        for i in range(0, len(prev_layer)):
            total = total + float(prev_layer[i].av) * prev_layer[i].weights[index]
            
            self.av = self.sigmoid(total)
            
    def sigmoid(self,inputs):
        sig = 1 / (1+math.exp(-inputs))
        return sig
    
    def __init__ (self, av, weights, position):
        #output
        self.av = av
        self.weights = weights
        # change to delta error and reposition
        self.dw = []
        self.grad_val = 0

In [4]:
i_layer = [neuron(0,2,0), neuron(0,2,1)]
h_layer = [neuron(0,1,0), neuron(0,1,1)]

# number of output values = number of output neurons
o_layer = [neuron(0,0,0)]

# originally bias(should be one value)
i_layer[0].weights = [-5.1,-4.3]
i_layer[1].weights = [5.2,4.0]
# Add Bias to the end of the weight list
h_layer[0].weights = [-6.0]
h_layer[1].weights = [6.5]

In [5]:
inputs = list(zip(x,y))
inputs

[(0.003896654, 0.598881259),
 (0.003896654, 0.598881259),
 (0.003518419, 0.599098716),
 (0.003518419, 0.59953363),
 (0.003140185, 0.600186002),
 (0.003140185, 0.601055831),
 (0.002761951, 0.602143117),
 (0.002761951, 0.603447861),
 (0.002383717, 0.604970062),
 (0.002383717, 0.60670972),
 (0.002005482, 0.608666835),
 (0.002005482, 0.610841408),
 (0.001627248, 0.613233438),
 (0.001627248, 0.615842925),
 (0.001249014, 0.618669869),
 (0.001249014, 0.621714271),
 (0.000870779, 0.62497613),
 (0.000870779, 0.628455446),
 (0.000492545, 0.63215222),
 (0.000492545, 0.63606645),
 (0.000114311, 0.640198138),
 (0.000180312, 0.644547019),
 (0.0, 0.649112297),
 (0.000395604, 0.65389265),
 (0.000676173, 0.658886227),
 (0.000906907, 0.664090655),
 (0.001152603, 0.669503036),
 (0.00147758, 0.675119955),
 (0.001945601, 0.680937484),
 (0.002619792, 0.686951183),
 (0.003562567, 0.693156112),
 (0.004835553, 0.699546829),
 (0.006499513, 0.706117403),
 (0.008614274, 0.712861418),
 (0.011238652, 0.719771982),


In [6]:
def feedForward(inputs):
    
    result = []
    
    # make more abstract
    for i in range(0,len(inputs)):
        i_layer[i].av = inputs[i]
    
    for i in range(0, len(h_layer)-1):
        #activation and transfer
        h_layer[i].multWeights(i_layer,i)
        
    #feed forward
    o_layer[0].multWeights(h_layer,0)
    
    result.append(o_layer[0].av)
    return result


for i in range(0, len(inputs)):
    print(i, feedForward(inputs[i]))



0 [0.0032046440509831864]
1 [0.0032046440509831864]
2 [0.0032022167307088883]
3 [0.003200428169873398]
4 [0.0031962382264644515]
5 [0.003192697050366271]
6 [0.0031868018207979213]
7 [0.0031815708276262075]
8 [0.0031740533408496484]
9 [0.0031672203533029265]
10 [0.0031581870441306824]
11 [0.003149862402052173]
12 [0.00313944021480188]
13 [0.003129753737897798]
14 [0.0031180867459367823]
15 [0.003107184238817225]
16 [0.003094429931294914]
17 [0.0030824694350230653]
18 [0.003068794837826719]
19 [0.0030559427187313803]
20 [0.0030415204299532863]
21 [0.003028147789380135]
22 [0.0030137320148574584]
23 [0.003000731493255942]
24 [0.0029871321475367033]
25 [0.002973183367731901]
26 [0.0029591136650095043]
27 [0.0029451294093503825]
28 [0.0029314145969274023]
29 [0.002918131474435814]
30 [0.0029054218402762896]
31 [0.0028934089132809583]
32 [0.002882199556138054]
33 [0.0028718868151719084]
34 [0.0028625526359700292]
35 [0.002854270718879032]
36 [0.00284710948957918]
37 [0.002841135121916005]
38

In [7]:
outputs = list(zip(vx,vy))
outputs

[(0.323456057, 0.979030201),
 (0.314982943, 1.0),
 (0.306509829, 0.979030201),
 (0.298036714, 1.0),
 (0.2895636, 0.979030201),
 (0.281090486, 1.0),
 (0.272617372, 0.979030201),
 (0.264144258, 1.0),
 (0.255671143, 0.979030201),
 (0.247198029, 1.0),
 (0.238724915, 0.979030201),
 (0.230251801, 1.0),
 (0.221778686, 0.979030201),
 (0.213305572, 1.0),
 (0.204832458, 0.979030201),
 (0.196359344, 1.0),
 (0.18788623, 0.979030201),
 (0.179413115, 1.0),
 (0.170940001, 0.979030201),
 (0.162466887, 1.0),
 (0.154004096, 0.975371024),
 (0.145572262, 0.989026927),
 (0.137191981, 0.957097423),
 (0.128883786, 0.963475062),
 (0.120668123, 0.966238025),
 (0.112565325, 0.965408492),
 (0.104595586, 0.961013023),
 (0.09677894, 0.953082522),
 (0.089135233, 0.9416522),
 (0.081684102, 0.926761531),
 (0.074444946, 0.908454205),
 (0.067436911, 0.886778075),
 (0.060678856, 0.8617851),
 (0.054189338, 0.833531277),
 (0.047986588, 0.802076579),
 (0.042088486, 0.767484875),
 (0.03651254, 0.72982386),
 (0.031275867, 0.

In [8]:
def BackPropagation(outputs):
    
    error = []
    lamb = 5
    lr = 2
    momentum = 3
    
    # error = (output - expected) * transfer_derivative(output)
    for i in range(len(o_layer)):
        e = outputs[i] - o_layer[i].av 
        error.append(e)
        
    for i in range(0,len(o_layer)):
        o_layer[i].grad_val = lamb * o_layer[i].av * (1 - o_layer[i].av) * error[i]
        
    for k in range(0,len(h_layer)):
        g = lamb * h_layer[k].av * (1 - h_layer[k].av)
        result = 0
        for i in range(0,len(o_layer)):
            result = result + (o_layer[i].grad_val * h_layer[k].weights[i])
        
        h_layer[k].grad_val = g * result
        
    for z in range(0, len(h_layer)):
        for y in range(0,len(o_layer)):
            #error backprop: dw(delta weight) -->  back propped error 
            h_layer[z].dw[y] = lr * o_layer[y].grad_val * h_layer[y].av + momentum * h_layer[z].dw[y]
    
    for y in range(0, len(i_layer)):
        for a in range(0,len(h_layer)-1):
            i_layer[y].dw[a] = lr * h_layer[a].grad_val * i_layer[a].av + momentum * i_layer[y].dw[a]
    
    for x in range(0, len(h_layer)):
        for i in range(0,len(o_layer)-1):
            h_layer[y].weights[i] = h_layer[x].dw[i] * h_layer[x].weights[i]
    # Why? input layer has no weights!
    for y in range(0, len(i_layer)):
        for a in range(0,len(h_layer)-1):
            i_layer[y].weights[a] = i_layer[y].dw[a] * i_layer[y].weights[a]
            
for i in range(0, len(outputs)):
    print(i, BackPropagation(outputs[i]))


IndexError: list index out of range